In [ ]:
!source test_speed_cpu.sh

In [2]:
import sys
sys.path.append('src')

In [9]:
import torch
import torch.neuron

from mmt import utils
from mmt.checkpoint import CheckpointRegistry
from mmt.decoder import Suggestion, ModelConfig, MMTDecoder

In [5]:
TEST_TEXT = 'Companies and LSPs can translate their content with the ModernMT service in many languages ' \
            'directly on their production environment thanks to our simple RESTful API .'
MODEL_DIR = 'model'
device=None

In [7]:
config = ModelConfig.load(MODEL_DIR)
builder = CheckpointRegistry.Builder()
for name, checkpoint_path in config.checkpoints:
    builder.register(name, checkpoint_path)
checkpoints = builder.build(device)
decoder = MMTDecoder(checkpoints, device=device, tuning_ops=config.tuning)
decoder._nn_needs_reset = True
translation = decoder.translate('en', 'it', [TEST_TEXT], None)[0]

print(translation.text)
print(translation.alignment)

2021-01-29 16:16:24 | INFO | fairseq.tasks.translation | [sl] dictionary: 32896 types
2021-01-29 16:16:24 | INFO | fairseq.tasks.translation | [tl] dictionary: 32896 types
2021-01-29 16:16:27 | INFO | Transformer | reset_time = 0.063, tune_time = 0.000, decode_time = 1.163


Le aziende e i LSP possono tradurre il loro contenuto con il servizio di modernità in molte lingue direttamente nel loro ambiente di produzione grazie alla nostra semplice API .
[(0, 0), (0, 1), (1, 2), (2, 4), (3, 5), (4, 6), (5, 7), (5, 8), (6, 9), (7, 10), (8, 11), (9, 13), (9, 14), (10, 12), (11, 15), (12, 16), (13, 17), (14, 18), (15, 19), (16, 20), (17, 22), (17, 23), (18, 21), (19, 24), (20, 24), (21, 25), (22, 26), (23, 27), (24, 28)]


## Attempt to compile the `decoder` object
It is using `fairseq` classes internally

In [16]:
print(f'Type of `decoder._model`: {type(decoder._model)}')
print(f'Type of `decoder._translator`: {type(decoder._translator)}')
print(f'Type of `decoder._tuner`: {type(decoder._tuner)}')

Type of `decoder._model`: <class 'fairseq.models.transformer.TransformerModel'>
Type of `decoder._translator`: <class 'fairseq.sequence_generator.SequenceGenerator'>
Type of `decoder._tuner`: <class 'mmt.tuning.Tuner'>


In [10]:
class MyGen(torch.nn.Module):
    def __init__(self, model, translator):
        super(MyGen, self).__init__()
        self.model = model
        self.translator = translator

    def forward(self, x):
        return self.translator.generate([self.model], x)

In [11]:
# Attempt to trace
batch, input_indexes, sentence_len = decoder._make_decode_batch([TEST_TEXT], prefix_lang=None)
my_gen = MyGen(decoder._model, decoder._translator)
torch.neuron.trace(my_gen, batch)

/home/ubuntu/work/torch_neuron_env/lib/python3.7/site-packages/fairseq/sequence_generator.py:229: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  int(self.max_len_a * src_len + self.max_len_b),
/home/ubuntu/work/torch_neuron_env/lib/python3.7/site-packages/fairseq/modules/sinusoidal_positional_embedding.py:71: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.weights is None or max_pos > self.weights.size(0):
/home/ubuntu/work/torch_neuron_env/lib/python3.7/site-packages/fairseq/modules/multihead_attention.py:152: TracerWarning: Converting a tensor t

/home/ubuntu/work/torch_neuron_env/lib/python3.7/site-packages/fairseq/sequence_generator.py:647: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  for i in range(bbsz_idx.size()[0]):
/home/ubuntu/work/torch_neuron_env/lib/python3.7/site-packages/fairseq/sequence_generator.py:653: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  sent = unfin_idx + cum_unfin[unfin_idx]
/home/ubuntu/work/torch_neuron_env/lib/python3.7/site-packages/fairseq/sequence_generator.py:657: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can'

RuntimeError: Only tensors, lists, tuples of tensors, or dictionary of tensors can be output from traced functions